# Лабораторная работа 2. Модели представления текстов

In [1]:
!pip install pymorphy2
!pip install -U pymorphy2-dicts-ru

     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 8.2 MB 19.8 MB/s 


In [1]:
import nltk
import os
import re
import pymorphy2
import json
import numpy as np
from nltk.tokenize import word_tokenize, sent_tokenize, SExprTokenizer
from nltk.corpus import stopwords
import time
import matplotlib.pyplot as plt
import pandas as pd
import string
import pandas as pd
import math
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim
import gensim.downloader
from gensim.models import KeyedVectors

In [2]:
nltk.download('punkt')
nltk.download('stopwords')

morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
TRAIN_FILE_NAME = 'train.json'
TEST_FILE_NAME = 'test.json'
ROUND_DIGITS = 3
ENCODING = 'utf-8'
ID = 'id'
SENTIMENT = 'sentiment'
SENTIMENTS = ['negative', 'positive', 'neutral']
TEXT = 'text'
POS = 'pos'
FREQUENCY = 'frequency'
NORMAL = 'normal'
IN_DICT = 'in_dict'
WORDFORM = 'wordform'
WORDFORMS =  'wordforms'
COUNT_UNIQUE = 'count_unique'
COUNT_ALL = 'count_all'
MAX_CHAR = 100000000
MIN_CHAR = 0

### Задание 1. Подготовка текстового корпуса
- Прочитайте описание задачи Sentiment analysis: https://www.kaggle.com/c/sentiment-analysis-in-russian.
- Скачайте файлы `train.json` и `test.json`: https://www.kaggle.com/c/sentiment-analysis-in-russian/data.
- Загрузите в ноутбук скачанные файлы, выведите информацию по ним.

In [6]:
def get_good_and_bad_train_examples(train_examples):
  beds = []
  goods = []
  for example in train_examples:
    if not (example[TEXT]) or not example[SENTIMENT] in SENTIMENTS:
      beds.append(example)
    else:
      goods.append(example)
  
  return goods, beds

In [7]:
def get_sentiments_dist(good_examples):
  dist = dict.fromkeys(SENTIMENTS, 0)

  for ex in good_examples:
    dist[ex[SENTIMENT]] = dist[ex[SENTIMENT]] + 1

  return dist

In [8]:
def get_lenghts(examples):
  min = MAX_CHAR
  avange = MIN_CHAR 
  max = MIN_CHAR
  for example in examples:
    min = len(example[TEXT]) if len(example[TEXT]) < min else min
    max = len(example[TEXT]) if len(example[TEXT]) > max else max
    avange += len(example[TEXT])
  print(f'max char {max}')
  print(f'min char {min}')
  print(f'avange char {round(avange/len(examples),ROUND_DIGITS)}')    

In [9]:
train = dict
time_start = 0
time_end = 0

with open(TRAIN_FILE_NAME , 'r', encoding = ENCODING) as fl:
  time_start = time.time() 
  train = json.load(fl)
  fl.close()
  time_end = time.time()

good_examples, bed_examples  = get_good_and_bad_train_examples(train)
sentiments_dist = get_sentiments_dist(good_examples)



print(f'volume {os.path.getsize(TRAIN_FILE_NAME)} bytes')
print(f'time read {round(time_end-time_start , ROUND_DIGITS)} sec')
print(f'count examples {len(train)}')
print(f'good examples {len(good_examples)}')
print(f'bad examples {len(bed_examples)}')
print(f'sentiments dist {sentiments_dist}')
get_lenghts(good_examples)

volume 59298269 bytes
time read 0.321 sec
count examples 8263
good examples 8263
bad examples 0
sentiments dist {'negative': 1434, 'positive': 2795, 'neutral': 4034}
max char 381498
min char 28
avange char 3911.85


In [10]:
test = dict
time_start = 0
time_end = 0

with open(TEST_FILE_NAME , 'r', encoding = ENCODING) as fl:
  time_start = time.time() 
  test = json.load(fl)
  fl.close()
  time_end = time.time()

print(f'volume {os.path.getsize(TRAIN_FILE_NAME)} bytes')
print(f'time read {round(time_end-time_start , ROUND_DIGITS)} sec')
print(f'count examples {len(test)}')
get_lenghts(test)

volume 59298269 bytes
time read 0.129 sec
count examples 2056
max char 320754
min char 35
avange char 4102.414


### Задание 2. Булевская модель
Напишите функцию `get_bool_model()` на входе которой текстовый корпус, на выходе – булевская матрица термин-документ.

В функции должны осуществляться следующие действия:
- все слова преобразуются в нормальную форму;
- удаляются стоп-слова;
- удаляются (или оставляются) слова заданных частей речи (список частей речи должен передаваться в виде параметра);
- удаляются слова, частотность которых во всем корпусе ниже заданного порога (параметр);
- создается словарь корпуса.

Нельзя использовать библиотечные функции `scikit-learn`.  
Можно использовать функции, разработанные в этой и предыдущей лабораторных работах.

In [10]:
def parse_text(text):
    arr = []
    tokens = word_tokenize(text.replace('\n',' '))
    for token in tokens:
        if not token in string.punctuation:
          morph_parse_best =  morph.parse(token)[0]
          word_struc = {}
          word_struc.update({WORDFORM : token})
          word_struc.update({NORMAL : morph_parse_best.normal_form})
          word_struc.update({POS : morph_parse_best.tag.POS})
          word_struc.update({IN_DICT : morph_parse_best.is_known})
          arr.append(word_struc)
            
    return arr

In [11]:
def count_frequency(word_norm, words_array):
    f = 0
    for word in words_array:
        f = f + 1 if word.get(NORMAL) == word_norm else f
    return f

def get_all_wordform(word_norm, words_array):
    wordforms = []
    for word in words_array:
        if word.get(NORMAL) == word_norm and word.get('wordform') not in wordforms:
            wordforms.append(word.get('wordform'))
    return wordforms
        

def get_dictionary(parsed_struct):
    dic = {}
    for word in parsed_struct:
        norm_word = {}
        if not word.get(NORMAL) in dic:
          frequency = count_frequency(word.get(NORMAL), parsed_struct) 
          norm_word[POS] = word[POS]
          norm_word.update({POS : word.get(POS)})
          norm_word.update({FREQUENCY : frequency})
          norm_word.update({WORDFORMS : get_all_wordform(word.get(NORMAL), parsed_struct)})
          dic.update({word.get(NORMAL) : norm_word})
    
    return dic

In [12]:
def get_bool_model(corpus, word_parts = [], p = 0):
    parsed_texts = []
    documents_words = []

    for text in corpus:
      parsed_text = parse_text(text[TEXT])
      documents_words.append(parsed_text)
      parsed_texts.extend(parsed_text)

    documents_dicts = [] 
    for doc_words in documents_words:
        documents_dicts.append(get_dictionary(doc_words))
    corpus_dict = get_dictionary(parsed_texts)

    clear_corpus_dict ={}
    for word_in_dict in corpus_dict:
      if not (corpus_dict[word_in_dict][FREQUENCY] < p or 
      word_in_dict in stopwords.words() or
      corpus_dict[word_in_dict][POS] in word_parts):
        clear_corpus_dict[word_in_dict] = corpus_dict[word_in_dict]

    term_matris = []
    for clear_corpus_dict_word in clear_corpus_dict:
      word_exist = []
      for doc_dict in documents_dicts:
        word_exist.append(True if clear_corpus_dict_word in doc_dict else False)
      term_matris.append(word_exist)

    

    return term_matris ,clear_corpus_dict

In [13]:
term_matris, corpus_dict = get_bool_model(good_examples[:5])

Постройте матрицу термин-документ для текстовых корпусов из первого задания.

In [14]:
term_doc = pd.DataFrame(data = term_matris, index = corpus_dict.keys())

In [15]:
term_doc

,0,1,2,3,4
досудебный,True,True,False,False,False
расследование,True,True,False,False,False
факт,True,True,True,False,True
покупка,True,False,True,False,False
енпф,True,False,False,False,False
...,...,...,...,...,...
информационный,False,False,False,False,True
обязательный,False,False,False,False,True
авторский,False,False,False,False,True
материал,False,False,False,False,True


### Задание 3. Модель TF-IDF
Напишите функцию `get_tfidf_model()` на входе которой текстовый корпус, на выходе – матрица термин-документ c TF-IDF-весами.

В функции должны осуществляться следующие действия:
- все слова преобразуются в нормальную форму;
- удаляются стоп-слова;
- удаляются (или оставляются) слова заданных частей речи (список частей речи должен передаваться в виде параметра);
- удаляются слова, частотность которых во всем корпусе ниже заданного порога (параметр);
- создается словарь корпуса;
- вычисляются глобальные IDF-веса и сохраняются в словарь;
- слова для документов взвешиваются в соответствии со схемой TF-IDF.

Нельзя использовать библиотечные функции `scikit-learn`.  
Можно использовать функции, разработанные в предыдущей лабораторной работе.

Постройте матрицу термин-документ для текстовых корпусов из первого задания.

In [16]:
def get_tfidf_model(corpus, word_parts = [], p = 0):
    parsed_texts = []
    documents_words = []

    for text in corpus:
      parsed_text = parse_text(text[TEXT])
      documents_words.append(parsed_text)
      parsed_texts.extend(parsed_text)

    corpus_dict = get_dictionary(parsed_texts)

    documents_dicts = [] 
    for doc_words in documents_words:
        documents_dicts.append(get_dictionary(doc_words))

    clear_corpus_dict = {}
    for word_in_dict in corpus_dict:
      if not (corpus_dict[word_in_dict][FREQUENCY] < p or 
      word_in_dict in stopwords.words() or
      corpus_dict[word_in_dict][POS] in word_parts):
        clear_corpus_dict[word_in_dict] = corpus_dict[word_in_dict]

    clear_documents_dicts = []
    for documents_dict in documents_dicts:
      clear_documents_dict = {}
      for word in documents_dict:
        if not (documents_dict[word][FREQUENCY] < p or 
        word in stopwords.words() or
        documents_dict[word][POS] in word_parts):
          clear_documents_dict[word] = documents_dict[word]
      clear_documents_dicts.append(clear_documents_dict)

    tfidf_matrix = []
    for clear_corpus_dict_word in clear_corpus_dict:
      tfidf_word = []
      counts_word = [] 
      for clear_documents_dict_n in clear_documents_dicts:  
        counts_word.append((clear_documents_dict_n[clear_corpus_dict_word][FREQUENCY]) if clear_corpus_dict_word in clear_documents_dict_n else 0 )
      df = len(corpus) - counts_word.count(0)
      
      for tf in counts_word:
        tfidf_word.append( tf * (np.log(len(corpus) / df ) + 1) if df != 0 else 0 )
      tfidf_matrix.append(tfidf_word)

    tfidf_matrix = np.fliplr(np.rot90(tfidf_matrix,3))


    norm_values = []
    for row in tfidf_matrix:
      norm_values.append(np.sqrt(sum([pow(x,2) for x in row])))

    norm_tfidf_matrix = []
    for doc_num, word_tf_idfs in enumerate(tfidf_matrix):
      norm_tfidf_matrix.append(
          [ tf_idf / norm_values[doc_num] for tf_idf in word_tf_idfs]
          )
    
    formated_texts = []

    for clear_document_dict in clear_documents_dicts:
      formated_text = []
      for word in clear_document_dict:
        for count in range(clear_document_dict[word][FREQUENCY]):
          formated_text.append(word)
      formated_texts.append(' '.join(formated_text))

    return norm_tfidf_matrix, clear_corpus_dict, clear_documents_dicts, formated_texts

In [17]:
tfidf_matrix, corpus_dict, docs_dicts, formated_texts  = get_tfidf_model(good_examples[:2])

In [18]:
tfidf_doc = pd.DataFrame(data = tfidf_matrix, columns= corpus_dict.keys())
tfidf_doc

,досудебный,расследование,факт,покупка,енпф,пакет,облигация,тоо,``,бузгул,аурум,'',начать,инициатива,национальный,банк,рк,сообщить,директор,департамент,защита,право,потребитель,финансовый,услуга,нацбанк,казахстан,александр,терентьев,основание,стать,обращение,письмо,25,ноябрь,2016,год,правоохранительный,орган,сделка,...,настоящий,полицейский,разыскивать,подозревать,местный,внутренний,статья,192,ук,разбой,создать,специальный,следственно-оперативный,группа,число,опытный,сотрудник,подразделение,криминальный,полиция,комплекс,оперативный,следственный,мероприятие,направить,установление,личность,нападать,задержание,ранее,данные,работать,штатный,режим,применение,усиленный,мера,безопасность,внутри,оцепить
0,0.078281,0.078281,0.078281,0.066271,0.265083,0.066271,0.066271,0.066271,0.265083,0.066271,0.066271,0.265083,0.039141,0.066271,0.265083,0.156562,0.039141,0.078281,0.066271,0.066271,0.066271,0.066271,0.066271,0.066271,0.066271,0.265083,0.039141,0.066271,0.132541,0.132541,0.132541,0.132541,0.066271,0.132541,0.132541,0.132541,0.132541,0.132541,0.132541,0.132541,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.032538,0.032538,0.032538,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032538,0.000000,0.000000,0.195226,0.032538,0.032538,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032538,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.110182,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.110182,0.055091,0.055091,0.110182,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.110182,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091,0.055091


Сравните результаты работы вашей функции (полученные веса) с результатами работы класса [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) из `scikit-learn`.

In [19]:
vectorizer = TfidfVectorizer(smooth_idf = False, tokenizer = word_tokenize, vocabulary = corpus_dict.keys())
X = vectorizer.fit_transform(formated_texts)
X.toarray()[:,:4]

array([[0.07828101, 0.07828101, 0.07828101, 0.06627063],
       [0.03253761, 0.03253761, 0.03253761, 0.        ]])

In [21]:
vectorizer.get_feature_names()[:4]


['досудебный', 'расследование', 'факт', 'покупка']

### Задание 4. Модель word2vec
Напишите функцию `get_word2vec_model()` на входе которой текстовый корпус и модель `word2vec`, на выходе – матрица термин-документ c word2vec-весами.

В функции должны осуществляться следующие действия:
- все слова преобразуются в нормальную форму (при необходимости – в зависимости от используемой модели);
- удаляются стоп-слова;
- удаляются (или оставляются) слова заданных частей речи (список частей речи должен передаваться в виде параметра);
- удаляются слова, частотность которых во всем корпусе ниже заданного порога (параметр);
- создается словарь корпуса;
- вычисляются веса word2vec для заданной модели.


Можно использовать библиотечные функции и функции, разработанные в лабораторных работах.

In [26]:
def load_model(model_name):
  if '.vec' in model_name:
    binary = False
  else:
    binary = True
    
  rus_model = gensim.models.KeyedVectors.load_word2vec_format(model_name, binary=binary)

  return rus_model

In [55]:
def get_word2vec_matrix(corpus, word2vec_model, word_parts = [], p = 0):
    parsed_texts = []
    documents_words = []

    for text in corpus:
      parsed_text = parse_text(text[TEXT])
      documents_words.append(parsed_text)
      parsed_texts.extend(parsed_text)

    corpus_dict = get_dictionary(parsed_texts)

    documents_dicts = [] 
    for doc_words in documents_words:
        documents_dicts.append(get_dictionary(doc_words))

    clear_corpus_dict = {}
    for word_in_dict in corpus_dict:
      if not (corpus_dict[word_in_dict][FREQUENCY] < p or 
      word_in_dict in stopwords.words() or
      corpus_dict[word_in_dict][POS] in word_parts):
        clear_corpus_dict[word_in_dict] = corpus_dict[word_in_dict]

    clear_documents_dicts = []
    for documents_dict in documents_dicts:
      clear_documents_dict = {}
      for word in documents_dict:
        if not (documents_dict[word][FREQUENCY] < p or 
        word in stopwords.words() or
        documents_dict[word][POS] in word_parts):
          clear_documents_dict[word] = documents_dict[word]
      clear_documents_dicts.append(clear_documents_dict)

    doc_vecs = []
    for doc_dict in clear_documents_dicts:
      doc_vec = np.zeros(300)
      count_word_in_model = 0 
      for word in doc_dict:
        if word + '_' + f'{doc_dict[word][POS]}' in word2vec_model:
          doc_vec += word2vec_model[word + '_' + doc_dict[word][POS]]
          count_word_in_model +=1
      doc_vecs.append([x / count_word_in_model for x in doc_vec])
  
    return doc_vecs, clear_corpus_dict, clear_documents_dicts

Постройте матрицу термин-документ для текстовых корпусов из первого задания.

In [56]:
model_name =  'ruscorpora_1_300_10.bin'
word2vec_model = load_model(model_name)


In [59]:
word2vec_matrix, corpus_dict, docs_dicts = get_word2vec_matrix(good_examples[:10], word2vec_model)

In [60]:
word2vec_doc = pd.DataFrame(data = word2vec_matrix)
word2vec_doc

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0.023934,-0.008687,-0.032956,0.009489,-0.011051,0.022231,0.008541,0.008891,0.015125,-0.022434,0.040881,-0.034065,-0.048911,-0.028715,0.027131,0.015274,0.039327,-0.002917,0.018240,0.019042,0.033503,0.046029,-0.064753,0.007301,0.021943,0.000255,0.028321,-0.040940,0.001860,0.015269,0.023772,0.008973,0.021884,-0.033943,0.026875,0.005891,-0.000102,0.004337,-0.021404,-0.006122,...,-0.038464,-0.001742,-0.055580,0.006630,-0.030240,0.033928,-0.009594,-0.016853,0.008064,0.002838,-0.002828,0.024906,-0.009971,0.027646,0.000718,0.001139,0.013961,0.000044,-0.001162,0.033633,-0.049601,-0.032006,0.007573,-0.009348,-0.000908,0.005303,-0.007305,0.000520,0.012230,0.015556,0.001519,0.049819,-0.002391,0.001870,-0.006077,-0.032091,0.047686,0.002281,-0.027877,-0.012895
1,0.015036,-0.004229,-0.013225,0.014743,-0.016161,0.020128,0.020240,-0.005424,0.028362,-0.035297,0.057704,-0.046386,-0.032482,-0.035669,0.031993,0.018120,0.032179,-0.002937,0.021734,0.019088,0.040698,0.045800,-0.050199,0.029392,0.030204,0.002456,0.031152,-0.021679,0.002172,-0.003611,0.008360,0.016211,0.036985,-0.023170,0.039775,0.019097,0.000036,0.015034,-0.011884,-0.007066,...,-0.029961,-0.002963,-0.051980,-0.006782,-0.025419,0.031979,-0.001920,-0.025209,0.005365,0.013871,0.008846,0.003181,-0.006680,0.011619,0.003871,0.011407,0.013134,-0.006842,0.007411,0.033247,-0.043761,-0.019066,0.016402,-0.017529,0.011391,0.006459,-0.018488,-0.029187,0.019849,0.018363,0.014277,0.041443,-0.016251,-0.023045,-0.016727,-0.019732,0.018296,0.029229,-0.037652,-0.013352
2,0.011523,-0.000783,-0.008847,0.025553,-0.019018,0.008022,0.021555,0.003517,0.005667,-0.024966,0.054336,-0.032567,-0.031399,-0.053273,0.038149,0.004329,0.033345,0.007069,0.014441,0.008578,0.021042,0.046600,-0.058240,0.028812,0.015021,-0.006633,0.009847,-0.035723,-0.011964,0.014600,0.019615,0.024337,0.045173,-0.015038,0.035876,0.032701,0.016396,0.017607,-0.016241,-0.014219,...,-0.039362,0.016867,-0.049764,0.002176,-0.014891,0.007749,-0.001943,-0.018513,-0.000295,0.010838,-0.002626,0.030176,0.007110,0.027782,-0.012436,0.005067,0.015891,0.010823,0.014089,0.031837,-0.053019,-0.023304,0.000604,-0.015899,0.000313,-0.003372,-0.005835,-0.018769,0.004009,0.004796,0.012829,0.018210,-0.009192,-0.005243,-0.012805,-0.037788,0.036030,-0.004580,-0.039948,-0.022735
3,0.023945,-0.016073,-0.026701,0.019765,-0.016460,0.018205,0.013256,-0.006509,0.012204,-0.039684,0.050101,-0.036860,-0.051542,-0.060824,0.035438,0.024177,0.038992,0.011186,0.020034,-0.002618,0.020342,0.038995,-0.067238,0.007815,0.026522,-0.021953,0.010269,-0.028972,0.003807,0.024210,0.025333,0.023896,0.027244,-0.018008,0.044002,0.023062,0.007556,0.019178,-0.021666,-0.002636,...,-0.020942,0.031402,-0.055523,-0.000316,-0.015748,0.007157,-0.014360,-0.015817,0.005617,0.007915,-0.009157,0.033233,0.013270,0.011814,-0.014705,0.010829,0.016775,0.012170,0.029538,0.032611,-0.055208,-0.040237,0.000552,-0.011457,-0.016888,0.005056,-0.002806,-0.008154,0.018899,0.010557,-0.006975,0.017054,-0.010664,-0.013977,-0.012582,-0.028057,0.037667,0.000141,-0.059651,-0.013532
4,0.016851,-0.013842,-0.036093,0.029677,0.001119,0.017064,-0.000674,0.012133,0.024788,-0.017186,0.064144,-0.043461,-0.054240,-0.051492,0.035236,0.011906,0.039868,0.006712,0.013308,0.019910,0.035384,0.062567,-0.064039,0.020747,0.006777,-0.005228,0.014398,-0.020186,-0.009258,0.013077,0.017375,0.035981,0.034271,-0.021778,0.033961,0.019734,0.024163,0.023585,-0.003024,-0.005662,...,-0.050699,-0.006087,-0.046444,0.004800,-0.016304,0.016843,-0.008833,-0.032562,0.009849,0.012026,-0.003066,0.028798,-0.003724,0.028405,-0.023565,0.001312,0.020046,-0.000748,0.009957,0.033241,-0.046485,-0.027229,-0.008810,-0.006455,-0.012436,0.015937,-0.013350,-0.015757,0.000103,-0.004149,0.020291,0.033